In [1]:
# REMOVED - NEW SYSTEM MOVED TO CORRECT POSITION AFTER CELL 7
# This cell was causing dependency issues by running before required functions were defined

In [2]:
def load_and_merge_ranking_data():
    """Load ESPN rankings and external ADP data, merge them for probability calculations"""
    
    # Load ESPN data (80% weight)
    espn_df = pd.read_csv('data/espn_projections_20250814.csv')
    
    # Load FantasyPros ADP data (20% weight)  
    adp_df = pd.read_csv('data/fantasypros_adp_20250815.csv')
    
    print(f"ESPN data: {len(espn_df)} players")
    print(f"ADP data: {len(adp_df)} players")
    
    # Standardize player names for matching
    def clean_name(name):
        return str(name).strip().lower().replace("'", "").replace(".", "")
    
    espn_df['clean_name'] = espn_df['player_name'].apply(clean_name)
    adp_df['clean_name'] = adp_df['PLAYER'].apply(clean_name)
    
    # Merge ESPN with ADP data
    merged_df = espn_df.merge(
        adp_df[['clean_name', 'RANK', 'ADP']], 
        on='clean_name', 
        how='left'
    )
    
    # Fill missing ADP ranks with ESPN rank + some penalty
    merged_df['adp_rank'] = merged_df['RANK'].fillna(merged_df['overall_rank'] + 50)
    merged_df['espn_rank'] = merged_df['overall_rank']
    
    print(f"Merged dataset: {len(merged_df)} players")
    print(f"Players with ADP data: {len(merged_df[merged_df['RANK'].notna()])} players")
    
    return merged_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week', 'espn_rank', 'adp_rank']]

def compute_softmax_scores(rank_series, tau=5.0):
    """Convert ranks to softmax scores with temperature tau (lower rank = higher score)"""
    scores = np.exp(-rank_series / tau)
    return scores

def compute_pick_probabilities(available_df, espn_weight=0.8, adp_weight=0.2, tau_espn=5.0, tau_adp=5.0):
    """
    Compute per-player probability of being picked next using weighted softmax over ESPN and ADP ranks
    """
    if len(available_df) == 0:
        return pd.Series(dtype=float)
    
    # Softmax scores for ESPN and ADP ranks
    espn_scores = compute_softmax_scores(available_df['espn_rank'], tau_espn)
    adp_scores = compute_softmax_scores(available_df['adp_rank'], tau_adp)
    
    # Weighted combination (80% ESPN, 20% ADP)
    combined_scores = espn_weight * espn_scores + adp_weight * adp_scores
    
    # Normalize to sum to 1 across available players
    probs = combined_scores / combined_scores.sum()
    return probs

def probability_gone_before_next_pick(available_df, player_name, picks_until_next_turn):
    """
    Compute probability a player is gone before your next turn using discrete survival calculation
    """
    if picks_until_next_turn <= 0:
        return 0.0
    
    survival_prob = 1.0
    current_available = available_df.copy()
    
    # Simulate each pick until our next turn
    for pick_step in range(picks_until_next_turn):
        if len(current_available) == 0:
            break
            
        # Get probabilities for current available players
        pick_probs = compute_pick_probabilities(current_available)
        
        # Find probability our target player gets picked this round
        player_mask = current_available['player_name'] == player_name
        if not player_mask.any():
            # Player already gone
            break
            
        p_pick_now = pick_probs[player_mask].iloc[0] if player_mask.any() else 0.0
        
        # Update survival probability
        survival_prob *= (1 - p_pick_now)
        
        # For simulation: remove the most likely pick (simplified approach)
        most_likely_idx = pick_probs.idxmax()
        current_available = current_available.drop(most_likely_idx)
    
    prob_gone = 1 - survival_prob
    return min(1.0, max(0.0, prob_gone))

# NOTE: ranking_df will be created after this cell is run by calling the function above

# Fantasy Football VBD Ranking Dashboard

Clean, minimal, interactive sortable table showing VBD rankings with probability intelligence.

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from IPython.core.display import display_html
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import warnings
warnings.filterwarnings('ignore')

# Enable widget support for proper rendering
try:
    from ipywidgets import interact, interactive, fixed
    import IPython.display as display_module
    print("✅ Widget support loaded successfully")
except ImportError:
    print("Warning: ipywidgets not fully available")

✅ Widget support loaded successfully


## 1. Core Calculation Engine (Preserved from Original)

In [4]:
def load_espn_data():
    """Load ESPN projections CSV data"""
    df = pd.read_csv('data/espn_projections_20250814.csv')
    return df

def load_vbd_data():
    """Load custom VBD rankings"""
    return pd.read_csv('draft_cheat_sheet.csv')

def merge_vbd_with_espn():
    """Merge VBD data with ESPN projections, using VBD rankings as primary"""
    vbd_df = load_vbd_data()
    espn_df = load_espn_data()
    
    print(f"VBD data loaded: {len(vbd_df)} players")
    print(f"ESPN data loaded: {len(espn_df)} players")
    
    # Start with VBD data as the foundation
    final_df = vbd_df.copy()
    
    # Standardize column names to match ESPN format
    final_df['player_name'] = final_df['Player']
    final_df['position'] = final_df['Position'] 
    final_df['overall_rank'] = final_df['Draft_Rank']
    final_df['salary_value'] = final_df['Custom_VBD']  # VBD values replace ESPN salary
    final_df['team'] = final_df['Team']
    final_df['bye_week'] = final_df['Bye']
    
    # Create position_rank from VBD data  
    position_ranks = []
    for _, row in final_df.iterrows():
        pos = row['position']
        # Count how many of this position come before this player in VBD ranking
        pos_rank = len(vbd_df[(vbd_df['Position'] == pos) & (vbd_df['Draft_Rank'] < row['overall_rank'])]) + 1
        position_ranks.append(f"{pos}{pos_rank}")
    
    final_df['position_rank'] = position_ranks
    
    # Keep only the ESPN-format columns
    final_df = final_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week']].copy()
    
    # Fill any remaining NaN values
    final_df = final_df.fillna(0)
    
    print(f"Successfully merged {len(final_df)} players from VBD data")
    print(f"Top 5 by VBD score:")
    print(final_df.head()[['player_name', 'position', 'overall_rank', 'salary_value']])
    
    # Debug: Show a few more to verify the merge worked
    print(f"\nVerification - All players using VBD values:")
    print(final_df[['player_name', 'overall_rank', 'salary_value']].head(10))
    
    return final_df

def calculate_availability(player_rank, draft_position, std_dev=3):
    """Calculate probability that a player is available at a given draft position"""
    if draft_position <= 0:
        return 1.0
    # Probability = 1 - CDF(draft_position, mean=player_rank, std=std_dev)
    prob = 1 - stats.norm.cdf(draft_position, loc=player_rank, scale=std_dev)
    return max(0, min(1, prob))

# Load the merged VBD + ESPN data
df = merge_vbd_with_espn()
print(f"\nLoaded {len(df)} players with VBD integration")
print("Sample of enhanced data:")
print(df.head())

VBD data loaded: 50 players
ESPN data loaded: 300 players
Successfully merged 50 players from VBD data
Top 5 by VBD score:
           player_name position  overall_rank  salary_value
0       Saquon Barkley       RB             1         131.3
1       Bijan Robinson       RB             2         127.3
2         Jahmyr Gibbs       RB             3         125.0
3        Ja'Marr Chase       WR             4         120.6
4  Christian McCaffrey       RB             5          96.9

Verification - All players using VBD values:
           player_name  overall_rank  salary_value
0       Saquon Barkley             1         131.3
1       Bijan Robinson             2         127.3
2         Jahmyr Gibbs             3         125.0
3        Ja'Marr Chase             4         120.6
4  Christian McCaffrey             5          96.9
5        Derrick Henry             6          95.8
6        De'Von Achane             7          92.7
7        Lamar Jackson             8          91.7
8     Justin

In [5]:
# Load the new merged ranking data (requires imports from Cell 3)
ranking_df = load_and_merge_ranking_data()
print(f"\nSample of merged ranking data:")
print(ranking_df[['player_name', 'espn_rank', 'adp_rank']].head(10))

ESPN data: 300 players
ADP data: 347 players
Merged dataset: 300 players
Players with ADP data: 252 players

Sample of merged ranking data:
           player_name  espn_rank  adp_rank
0        Ja'Marr Chase          1       1.0
1       Bijan Robinson          2       2.0
2     Justin Jefferson          3       5.0
3       Saquon Barkley          4       3.0
4         Jahmyr Gibbs          5       4.0
5          CeeDee Lamb          6       6.0
6  Christian McCaffrey          7      11.0
7           Puka Nacua          8       7.0
8         Malik Nabers          9       8.0
9    Amon-Ra St. Brown         10       9.0


## 2. Enhanced Data Processing for Dashboard

In [6]:
def calculate_player_metrics(df, my_picks=None, drafted_players=None, current_pick=1):
    """Calculate enhanced metrics for each player with dynamic draft position logic"""
    if my_picks is None:
        my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default pick positions
    if drafted_players is None:
        drafted_players = set()
    
    # Calculate dynamic draft positions
    next_pick = None
    picks_to_next = None
    pick_after_next = None
    
    for pick in my_picks:
        if pick >= current_pick:
            next_pick = pick
            picks_to_next = pick - current_pick
            # Find pick after next
            remaining_picks = [p for p in my_picks if p > next_pick]
            pick_after_next = remaining_picks[0] if remaining_picks else None
            break
    
    print(f"DRAFT POSITION LOGIC:")
    print(f"Current pick: {current_pick}")
    print(f"Your next pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Your pick after that: {pick_after_next}")
    print("=" * 50)
    
    enhanced_data = []
    
    # Calculate position scarcity data using VBD values
    position_counts = {}
    for pos in ['QB', 'RB', 'WR', 'TE']:
        pos_players = df[df['position'] == pos]
        # Count players above median VBD threshold (Custom_VBD >= 40)
        elite_count = len(pos_players[pos_players['salary_value'] >= 40])
        position_counts[pos] = elite_count
    
    for _, player in df.iterrows():
        if player['player_name'] in drafted_players:
            continue  # Skip drafted players
            
        # Use Draft_Rank instead of ESPN overall_rank for availability calculations
        player_rank = player['overall_rank']  # This is now Draft_Rank from VBD data
        custom_vbd = player['salary_value']   # This is now Custom_VBD from VBD data
        position = player['position']
        
        # Calculate probabilities at dynamic picks
        prob_at_next_pick = calculate_availability(player_rank, next_pick) if next_pick else 0.0
        prob_at_pick_after = calculate_availability(player_rank, pick_after_next) if pick_after_next else 0.0
        
        # Calculate Decision Score using Custom_VBD × Probability at next pick
        decision_score = custom_vbd * prob_at_next_pick
        
        # Calculate median pick using VBD Draft_Rank (50% probability point)
        median_pick = player_rank
        
        # Calculate 10th and 90th percentile picks for range using VBD ranking
        p10_pick = max(1, stats.norm.ppf(0.9, loc=player_rank, scale=3))
        p90_pick = stats.norm.ppf(0.1, loc=player_rank, scale=3)
        
        # Calculate opportunity cost (next best VBD value at position if wait)
        pos_players = df[df['position'] == position]
        # Sort by VBD ranking (overall_rank which is now Draft_Rank)
        better_players = pos_players[pos_players['overall_rank'] > player_rank].sort_values('overall_rank')
        if len(better_players) > 0:
            next_best_vbd = better_players.iloc[0]['salary_value'] if len(better_players) > 0 else 0
            opportunity_cost = max(0, custom_vbd - next_best_vbd)  # Cost of waiting vs drafting now
        else:
            opportunity_cost = 0
        
        # Positional scarcity calculation using VBD position ranking
        pos_rank_num = int(player['position_rank'].replace(position, '')) if player['position_rank'] else 999
        if position in position_counts:
            remaining_elite = max(0, position_counts[position] - pos_rank_num + 1)
            scarcity_text = f"{position}{pos_rank_num} ({remaining_elite} of {position_counts[position]} left)"
        else:
            scarcity_text = f"{position}{pos_rank_num}"
        
        # Generate decision notes with strategic guidance
        decision_notes = ""
        if prob_at_next_pick > 0.8:
            decision_notes = f"SAFE - Available at pick {next_pick}"
        elif prob_at_pick_after and prob_at_pick_after > 0.7:
            decision_notes = f"WAIT - Target at pick {pick_after_next}"
        elif prob_at_next_pick > 0.3:
            decision_notes = f"DRAFT NOW - Risky to wait"
        else:
            decision_notes = f"REACH - Must draft at pick {next_pick} to secure"
        
        # Add value consideration
        if custom_vbd >= 100:
            decision_notes = "ELITE - " + decision_notes
        elif custom_vbd >= 70:
            decision_notes = "STRONG - " + decision_notes
        elif custom_vbd >= 40:
            decision_notes = "SOLID - " + decision_notes
        else:
            decision_notes = "DEPTH - " + decision_notes
        
        # Generate sparkline data using VBD Draft_Rank (availability across next 16 picks)
        sparkline_picks = list(range(max(1, player_rank - 5), player_rank + 12))
        sparkline_probs = [calculate_availability(player_rank, pick) for pick in sparkline_picks]
        
        enhanced_data.append({
            'player_name': player['player_name'],
            'position': player['position'],
            'position_rank': player['position_rank'],
            'overall_rank': player['overall_rank'],  # This is now VBD Draft_Rank
            'team': player['team'],
            'salary_value': player['salary_value'],  # This is now Custom_VBD
            'bye_week': player['bye_week'],
            'current_pick': current_pick,
            'next_pick': next_pick,
            'picks_to_next': picks_to_next,
            'pick_after_next': pick_after_next,
            'prob_at_next_pick': prob_at_next_pick,
            'prob_at_pick_after': prob_at_pick_after,
            'decision_score': decision_score,  # Now uses Custom_VBD × Probability at next pick
            'opportunity_cost': opportunity_cost,
            'scarcity_badge': scarcity_text,
            'decision_notes': decision_notes,
            'median_pick': median_pick,
            'p10_pick': int(p10_pick),
            'p90_pick': int(max(1, p90_pick)),
            'pick_range': f"{int(max(1, p90_pick))}-{int(p10_pick)}",
            'sparkline_picks': sparkline_picks,
            'sparkline_probs': sparkline_probs
        })
    
    enhanced_df = pd.DataFrame(enhanced_data)
    
    print(f"SUCCESS: All {len(enhanced_df)} players now use dynamic draft position logic")
    print(f"Probabilities calculated for next pick ({next_pick}) and pick after ({pick_after_next})")
    
    return enhanced_df

# Calculate metrics with dynamic draft position logic
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # 8-team league, pick 8
current_draft_pick = 1  # Current pick in the draft

enhanced_df = calculate_player_metrics(df, my_picks=my_draft_picks, current_pick=current_draft_pick)
print(f"\nEnhanced data for {len(enhanced_df)} available players with dynamic draft logic")
print("\nTop 10 players by VBD Rank:")
top_vbd = enhanced_df.sort_values('overall_rank').head(10)
print(top_vbd[['player_name', 'position', 'overall_rank', 'salary_value', 'prob_at_next_pick', 'prob_at_pick_after', 'decision_notes']].round(2))

DRAFT POSITION LOGIC:
Current pick: 1
Your next pick: 8 (in 7 picks)
Your pick after that: 17
SUCCESS: All 50 players now use dynamic draft position logic
Probabilities calculated for next pick (8) and pick after (17)

Enhanced data for 50 available players with dynamic draft logic

Top 10 players by VBD Rank:
           player_name position  overall_rank  salary_value  \
0       Saquon Barkley       RB             1         131.3   
1       Bijan Robinson       RB             2         127.3   
2         Jahmyr Gibbs       RB             3         125.0   
3        Ja'Marr Chase       WR             4         120.6   
4  Christian McCaffrey       RB             5          96.9   
5        Derrick Henry       RB             6          95.8   
6        De'Von Achane       RB             7          92.7   
7        Lamar Jackson       QB             8          91.7   
8     Justin Jefferson       WR             9          88.3   
9           Josh Allen       QB            10          88.

In [7]:
def calculate_player_metrics_new_system(ranking_df, vbd_df, my_picks=None, drafted_players=None, current_pick=1):
    """Calculate enhanced metrics using new 80% ESPN + 20% ADP probability system"""
    if my_picks is None:
        my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default pick positions
    if drafted_players is None:
        drafted_players = set()
    
    # Get next pick logic
    next_pick = None
    picks_to_next = None
    pick_after_next = None
    
    for pick in my_picks:
        if pick >= current_pick:
            next_pick = pick
            picks_to_next = pick - current_pick
            remaining_picks = [p for p in my_picks if p > next_pick]
            pick_after_next = remaining_picks[0] if remaining_picks else None
            break
    
    print(f"NEW SYSTEM - DRAFT POSITION LOGIC:")
    print(f"Current pick: {current_pick}")
    print(f"Your next pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Your pick after that: {pick_after_next}")
    print("=" * 50)
    
    # Filter out drafted players from ranking data
    available_ranking_df = ranking_df[~ranking_df['player_name'].isin(drafted_players)].copy()
    
    # Merge with VBD data for VBD scores
    vbd_lookup = dict(zip(vbd_df['Player'], vbd_df['Custom_VBD']))
    available_ranking_df['vbd_score'] = available_ranking_df['player_name'].map(vbd_lookup)
    available_ranking_df['vbd_score'] = available_ranking_df['vbd_score'].fillna(0)
    
    enhanced_data = []
    
    for _, player in available_ranking_df.iterrows():
        player_name = player['player_name']
        espn_rank = player['espn_rank']
        adp_rank = player['adp_rank']
        vbd_score = player['vbd_score']
        
        # Calculate NEW probabilities using discrete survival method
        prob_gone_by_next = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_next
        ) if picks_to_next > 0 else 0.0
        
        prob_available_at_next = 1 - prob_gone_by_next
        
        # Calculate probability gone by pick after next
        picks_to_after = (pick_after_next - current_pick) if pick_after_next else 0
        prob_gone_by_after = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_after
        ) if picks_to_after > 0 else 0.0
        
        prob_available_at_after = 1 - prob_gone_by_after
        
        # Decision score using VBD × availability probability
        decision_score = vbd_score * prob_available_at_next
        
        # Get immediate next-pick probability for this player
        immediate_pick_probs = compute_pick_probabilities(available_ranking_df)
        player_idx = available_ranking_df.index[available_ranking_df['player_name'] == player_name][0]
        immediate_prob = immediate_pick_probs.loc[player_idx]
        
        # Decision logic
        if prob_available_at_next > 0.8:
            decision_notes = f"SAFE - {prob_available_at_next:.0%} chance available at pick {next_pick}"
        elif prob_available_at_after and prob_available_at_after > 0.7:
            decision_notes = f"WAIT - Target at pick {pick_after_next}"
        elif prob_available_at_next > 0.3:
            decision_notes = f"DRAFT NOW - Only {prob_available_at_next:.0%} chance available later"
        else:
            decision_notes = f"REACH - Must draft at pick {next_pick} to secure"
        
        # Add VBD tier
        if vbd_score >= 100:
            decision_notes = "ELITE - " + decision_notes
        elif vbd_score >= 70:
            decision_notes = "STRONG - " + decision_notes
        elif vbd_score >= 40:
            decision_notes = "SOLID - " + decision_notes
        else:
            decision_notes = "DEPTH - " + decision_notes
        
        enhanced_data.append({
            'player_name': player_name,
            'position': player['position'],
            'position_rank': player['position_rank'],
            'espn_rank': int(espn_rank),
            'adp_rank': int(adp_rank),
            'vbd_score': vbd_score,
            'team': player['team'],
            'bye_week': player['bye_week'],
            'current_pick': current_pick,
            'next_pick': next_pick,
            'picks_to_next': picks_to_next,
            'pick_after_next': pick_after_next,
            'prob_available_at_next': prob_available_at_next,
            'prob_available_at_after': prob_available_at_after,
            'immediate_pick_prob': immediate_prob,
            'decision_score': decision_score,
            'decision_notes': decision_notes
        })
    
    enhanced_df = pd.DataFrame(enhanced_data)
    
    print(f"SUCCESS: Enhanced data calculated for {len(enhanced_df)} available players")
    print(f"Using new 80% ESPN + 20% ADP discrete survival probability system")
    
    return enhanced_df

# Test the new system
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]
current_draft_pick = 1
drafted_players_list = set()  # Empty for now

# Load VBD data for scoring
vbd_data = load_vbd_data()

# Calculate using new system
new_enhanced_df = calculate_player_metrics_new_system(
    ranking_df, vbd_data, 
    my_picks=my_draft_picks, 
    current_pick=current_draft_pick,
    drafted_players=drafted_players_list
)

print(f"\nTop 10 players by ESPN rank using NEW probability system:")
top_new = new_enhanced_df.sort_values('espn_rank').head(10)
print(top_new[['player_name', 'espn_rank', 'adp_rank', 'vbd_score', 'prob_available_at_next', 'decision_notes']].round(3))

NEW SYSTEM - DRAFT POSITION LOGIC:
Current pick: 1
Your next pick: 8 (in 7 picks)
Your pick after that: 17
SUCCESS: Enhanced data calculated for 300 available players
Using new 80% ESPN + 20% ADP discrete survival probability system

Top 10 players by ESPN rank using NEW probability system:
           player_name  espn_rank  adp_rank  vbd_score  \
0        Ja'Marr Chase          1         1      120.6   
1       Bijan Robinson          2         2      127.3   
2     Justin Jefferson          3         5       88.3   
3       Saquon Barkley          4         3      131.3   
4         Jahmyr Gibbs          5         4      125.0   
5          CeeDee Lamb          6         6       74.4   
6  Christian McCaffrey          7        11       96.9   
7           Puka Nacua          8         7       67.7   
8         Malik Nabers          9         8       57.5   
9    Amon-Ra St. Brown         10         9       62.1   

   prob_available_at_next                                     decisio

In [8]:
# Create unified viz_df for visualizations
# This creates a single data source that all visualization cells can use

# Use globals() instead of locals() to access variables from other cells
try:
    # Try new system first (if available)
    if 'new_enhanced_df' in globals() and new_enhanced_df is not None and len(new_enhanced_df) > 0:
        viz_df = new_enhanced_df.copy()
        print("✅ Using new_enhanced_df (80% ESPN + 20% ADP system) for visualizations")
    # Fall back to enhanced_df 
    elif 'enhanced_df' in globals() and enhanced_df is not None and len(enhanced_df) > 0:
        viz_df = enhanced_df.copy()
        print("✅ Using enhanced_df (original system) for visualizations")
    else:
        viz_df = None
        print("⚠️ No enhanced data available. Please run cells 1-7 to load and calculate player metrics.")
        print(f"Available variables: {[var for var in globals().keys() if 'df' in var.lower()]}")
        
    if viz_df is not None:
        print(f"📊 Visualization data ready: {len(viz_df)} players")
        print(f"Available columns: {list(viz_df.columns)}")
        
except Exception as e:
    viz_df = None
    print(f"❌ Error creating viz_df: {e}")
    print("Please run the data preparation cells first")

✅ Using new_enhanced_df (80% ESPN + 20% ADP system) for visualizations
📊 Visualization data ready: 300 players
Available columns: ['player_name', 'position', 'position_rank', 'espn_rank', 'adp_rank', 'vbd_score', 'team', 'bye_week', 'current_pick', 'next_pick', 'picks_to_next', 'pick_after_next', 'prob_available_at_next', 'prob_available_at_after', 'immediate_pick_prob', 'decision_score', 'decision_notes']


In [9]:
def create_position_filter_widget(enhanced_df):
    """Create interactive position filter widget for the enhanced data"""
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Validate input data
    if enhanced_df is None or len(enhanced_df) == 0:
        raise ValueError("DataFrame is empty or None")
    
    # Get available positions
    positions = ['All'] + sorted(enhanced_df['position'].unique().tolist())
    
    # Create widgets with explicit layouts
    position_dropdown = widgets.Dropdown(
        options=positions,
        value='All',
        description='Position:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='150px')
    )
    
    # Get available sort columns
    sort_columns = ['overall_rank', 'espn_rank', 'vbd_score', 'salary_value', 'prob_available_at_next', 'prob_at_next_pick', 'decision_score']
    available_sort_cols = [col for col in sort_columns if col in enhanced_df.columns]
    
    sort_dropdown = widgets.Dropdown(
        options=available_sort_cols,
        value=available_sort_cols[0] if available_sort_cols else 'player_name',
        description='Sort by:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px')
    )
    
    # Output widget for the table
    output = widgets.Output()
    
    def update_table(*args):
        with output:
            clear_output(wait=True)
            try:
                # Filter data
                if position_dropdown.value == 'All':
                    filtered_df = enhanced_df.copy()
                else:
                    filtered_df = enhanced_df[enhanced_df['position'] == position_dropdown.value]
                
                # Sort data
                sort_ascending = sort_dropdown.value in ['player_name', 'position']
                filtered_df = filtered_df.sort_values(sort_dropdown.value, ascending=sort_ascending)
                
                # Display table
                display_cols = ['player_name', 'position', 'team']
                
                # Add available columns based on what's in the dataframe
                if 'vbd_score' in filtered_df.columns:
                    display_cols.append('vbd_score')
                elif 'salary_value' in filtered_df.columns:
                    display_cols.append('salary_value')
                
                if 'prob_available_at_next' in filtered_df.columns:
                    display_cols.append('prob_available_at_next')
                elif 'prob_at_next_pick' in filtered_df.columns:
                    display_cols.append('prob_at_next_pick')
                
                if 'decision_notes' in filtered_df.columns:
                    display_cols.append('decision_notes')
                
                display_df = filtered_df[display_cols].head(15)
                print(f"Showing top 15 {position_dropdown.value if position_dropdown.value != 'All' else 'players'} sorted by {sort_dropdown.value}")
                print("=" * 80)
                print(display_df.to_string(index=False))
                
            except Exception as e:
                print(f"Error updating table: {e}")
                import traceback
                traceback.print_exc()
    
    # Set up event handlers BEFORE displaying
    position_dropdown.observe(update_table, names='value')
    sort_dropdown.observe(update_table, names='value')
    
    # Create layout
    controls = widgets.HBox([position_dropdown, sort_dropdown])
    
    # Display everything at once to prevent display issues
    print("INTERACTIVE POSITION FILTER")
    print("Use dropdowns to filter by position and change sort order")
    print("="*80)
    
    # Use VBox to ensure proper vertical layout
    widget_container = widgets.VBox([controls, output])
    display(widget_container)
    
    # Trigger initial display
    update_table()
    
    return position_dropdown, sort_dropdown

# Create the widget with proper error handling
print("✅ Creating position filter widget")
try:
    # Check for data availability using globals()
    if 'new_enhanced_df' in globals() and new_enhanced_df is not None and len(new_enhanced_df) > 0:
        position_widget, sort_widget = create_position_filter_widget(new_enhanced_df)
    elif 'enhanced_df' in globals() and enhanced_df is not None and len(enhanced_df) > 0:
        position_widget, sort_widget = create_position_filter_widget(enhanced_df)
    elif 'viz_df' in globals() and viz_df is not None and len(viz_df) > 0:
        position_widget, sort_widget = create_position_filter_widget(viz_df)
    else:
        print("⚠️ No enhanced data available. Please run the data preparation cells first.")
        print("Available DataFrames:")
        for var_name in ['new_enhanced_df', 'enhanced_df', 'viz_df']:
            if var_name in globals():
                df = globals()[var_name]
                print(f"  {var_name}: {len(df) if df is not None else 'None'} rows")
        
except Exception as e:
    print(f"❌ Error creating position filter: {e}")
    import traceback
    traceback.print_exc()

✅ Creating position filter widget
INTERACTIVE POSITION FILTER
Use dropdowns to filter by position and change sort order


In [10]:
def create_styled_table(enhanced_df, limit=15):
    """Create a styled pandas table with color gradients for better visual clarity"""
    # Sort by VBD score (highest first)
    vbd_col = 'vbd_score' if 'vbd_score' in enhanced_df.columns else 'salary_value'
    df_display = enhanced_df.sort_values(vbd_col, ascending=False).head(limit).copy()
    
    # Create simplified table for styling - just VBD ranking
    try:
        styled_df = pd.DataFrame({
            'Player': df_display['player_name'],
            'Pos': df_display['position'],
            'Team': df_display['team'],
            'VBD': df_display[vbd_col].round(1),
            'Next %': ((df_display['prob_available_at_next'] if 'prob_available_at_next' in df_display.columns else df_display['prob_at_next_pick']) * 100).round(0),
            'Decision': df_display['decision_notes'].str.split(' - ').str[-1]
        })
    except Exception as e:
        print(f"Error creating styled table: {e}")
        return None
    
    # Apply styling functions
    def color_vbd(val):
        if val >= 100:
            return 'background-color: #2ecc71; color: white; font-weight: bold'
        elif val >= 70:
            return 'background-color: #27ae60; color: white'
        elif val >= 40:
            return 'background-color: #f39c12; color: white'
        else:
            return 'background-color: #95a5a6; color: white'
    
    def color_probability(val):
        if val >= 80:
            return 'background-color: #3498db; color: white'
        elif val >= 50:
            return 'background-color: #5dade2; color: white'
        elif val >= 30:
            return 'background-color: #aed6f1'
        else:
            return 'background-color: #e74c3c; color: white'
    
    def color_decision(val):
        if 'SAFE' in val or 'WAIT' in val:
            return 'background-color: #2ecc71; color: white'
        elif 'DRAFT NOW' in val:
            return 'background-color: #f39c12; color: white; font-weight: bold'
        elif 'REACH' in val:
            return 'background-color: #e74c3c; color: white; font-weight: bold'
        else:
            return ''
    
    # Apply styles
    styled = styled_df.style \
        .applymap(color_vbd, subset=['VBD']) \
        .applymap(color_probability, subset=['Next %']) \
        .applymap(color_decision, subset=['Decision']) \
        .format({'VBD': '{:.1f}', 'Next %': '{:.0f}'}) \
        .set_properties(**{'text-align': 'center'}) \
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#34495e'), 
                                         ('color', 'white'),
                                         ('font-weight', 'bold'),
                                         ('text-align', 'center')]},
            {'selector': 'td', 'props': [('padding', '8px')]}
        ])
    
    print("TOP PLAYERS BY VBD SCORE")
    print("Green=High VBD | Blue=High Availability | Orange=Act Now | Red=Reach")
    print("="*80)
    
    return styled

# Execute styled table creation
try:
    if 'viz_df' in globals() and viz_df is not None:
        print("🎨 CREATING VBD-RANKED TABLE")
        styled_table = create_styled_table(viz_df, limit=15)
        if styled_table is not None:
            display(styled_table)
    else:
        print("❌ ERROR: No visualization data available")
        print("Please run the data setup cells first")
        
except Exception as e:
    print(f"❌ Error creating styled table: {e}")
    print("Please ensure the data preparation cells have been run successfully")

🎨 CREATING VBD-RANKED TABLE
TOP PLAYERS BY VBD SCORE
Green=High VBD | Blue=High Availability | Orange=Act Now | Red=Reach


,Player,Pos,Team,VBD,Next %,Decision
3,Saquon Barkley,RB,PHI,131.3,54,Only 54% chance available later
1,Bijan Robinson,RB,ATL,127.3,70,Only 70% chance available later
4,Jahmyr Gibbs,RB,DET,125.0,49,Only 49% chance available later
0,Ja'Marr Chase,WR,CIN,120.6,82,82% chance available at pick 8
6,Christian McCaffrey,RB,SF,96.9,49,Only 49% chance available later
10,Derrick Henry,RB,BAL,95.8,72,Only 72% chance available later
17,De'Von Achane,RB,MIA,92.7,91,91% chance available at pick 8
28,Lamar Jackson,QB,BAL,91.7,99,99% chance available at pick 8
2,Justin Jefferson,WR,MIN,88.3,63,Only 63% chance available later
27,Josh Allen,QB,BUF,88.2,98,98% chance available at pick 8


In [11]:
def create_clean_probability_heatmap(enhanced_df, top_n=15, max_pick=30):
    """Create an enhanced heatmap showing availability probabilities across draft picks with user draft positions highlighted"""
    import plotly.graph_objects as go
    import plotly.express as px
    
    # User's draft picks
    user_picks = [8, 17, 32, 41, 56, 65, 80, 89]
    
    # Get top players by VBD score or ranking
    sort_col = 'vbd_score' if 'vbd_score' in enhanced_df.columns else 'salary_value'
    top_players = enhanced_df.nlargest(top_n, sort_col)
    
    # Create probability matrix for picks 1 through max_pick
    picks = list(range(1, max_pick + 1))
    
    # Enhanced player labels with position and VBD score
    player_labels = []
    for _, player in top_players.iterrows():
        position = player.get('position', 'N/A')
        vbd_score = player.get('vbd_score', player.get('salary_value', 0))
        player_name = player['player_name']
        label = f"{player_name} ({position}) - VBD: {vbd_score:.1f}"
        player_labels.append(label)
    
    # Calculate availability probability for each player at each pick
    prob_matrix = []
    hover_text = []
    for i, (_, player) in enumerate(top_players.iterrows()):
        player_rank = player.get('espn_rank', player.get('overall_rank', 1))
        row_probs = []
        row_hover = []
        for pick in picks:
            # Use normal distribution for availability probability
            prob = 1 - stats.norm.cdf(pick, loc=player_rank, scale=3)
            prob = max(0, min(1, prob))
            prob_pct = prob * 100
            row_probs.append(prob_pct)
            
            # Create decision guidance for hover
            if prob > 0.7:
                guidance = "SAFE"
            elif prob > 0.3:
                guidance = "RISKY" 
            else:
                guidance = "DRAFT NOW"
            
            hover_info = f"<b>{player['player_name']}</b><br>Pick {pick}: {prob_pct:.1f}% available<br>{guidance}<extra></extra>"
            row_hover.append(hover_info)
        
        prob_matrix.append(row_probs)
        hover_text.append(row_hover)
    
    # Create heatmap with improved color scheme
    fig = go.Figure(data=go.Heatmap(
        z=prob_matrix,
        x=picks,
        y=player_labels,
        colorscale=[
            [0.0, '#e74c3c'],    # Red for low availability (<30%)
            [0.3, '#f39c12'],    # Yellow for risky (30-70%)
            [0.7, '#2ecc71'],    # Green for safe (>70%)
            [1.0, '#27ae60']     # Dark green for very safe
        ],
        colorbar=dict(
            title=dict(text="Availability %", side="right"),
            tickmode="linear",
            tick0=0,
            dtick=20,
            ticksuffix='%'
        ),
        hovertemplate='%{text}',
        text=hover_text
    ))
    
    # Add vertical lines for user's draft picks
    for pick in user_picks:
        if pick <= max_pick:
            fig.add_vline(
                x=pick,
                line_dash="dash",
                line_color="white",
                line_width=2,
                annotation_text=f"Your Pick {pick}",
                annotation_position="top"
            )
    
    fig.update_layout(
        title="Enhanced Player Availability Heatmap - Top Players by VBD Score<br><sub>Green=Safe to Wait | Yellow=Risky | Red=Draft Now | White lines=Your picks</sub>",
        xaxis_title="Draft Pick",
        yaxis_title="Player (Position) - VBD Score",
        height=700,
        width=1200,
        font=dict(size=11),
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=2,
            range=[0.5, max_pick + 0.5]
        )
    )
    
    fig.show()
    return fig

def create_decision_matrix(enhanced_df, my_next_pick=8):
    """Create a decision matrix visualization"""
    import plotly.express as px
    
    # Filter to players with meaningful VBD scores
    vbd_col = 'vbd_score' if 'vbd_score' in enhanced_df.columns else 'salary_value'
    prob_col = 'prob_available_at_next' if 'prob_available_at_next' in enhanced_df.columns else 'prob_at_next_pick'
    
    decision_df = enhanced_df[enhanced_df[vbd_col] > 0].copy()
    
    # Create scatter plot
    fig = px.scatter(
        decision_df,
        x=prob_col,
        y=vbd_col,
        color='position',
        size=vbd_col,
        hover_data=['player_name', 'team'],
        title=f"Decision Matrix - Value vs Availability at Pick {my_next_pick}",
        labels={
            prob_col: "Probability Available at Next Pick",
            vbd_col: "VBD Score"
        }
    )
    
    # Add quadrant lines
    fig.add_hline(y=70, line_dash="dash", line_color="gray", annotation_text="High Value Threshold")
    fig.add_vline(x=0.5, line_dash="dash", line_color="gray", annotation_text="50% Availability")
    
    fig.update_layout(height=600, width=1000)
    fig.show()
    return fig

# Create the improved visualizations
try:
    if 'viz_df' in globals() and viz_df is not None:
        print("🎨 CREATING ENHANCED HEATMAP WITH ALL IMPROVEMENTS")
        clean_heatmap = create_clean_probability_heatmap(viz_df, top_n=15, max_pick=30)
        
        print("\n📊 CREATING DECISION MATRIX")
        decision_matrix = create_decision_matrix(viz_df, my_next_pick=8)
        
    else:
        print("❌ ERROR: No visualization data available")
        print("Please run the data setup cells first")
        
except Exception as e:
    print(f"❌ Error creating clean visualizations: {e}")
    print("Please ensure the data preparation cells have been run successfully")

🎨 CREATING ENHANCED HEATMAP WITH ALL IMPROVEMENTS



📊 CREATING DECISION MATRIX
